In [15]:
import json
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam
import pandas as pd

def is_chinese(string):
    for ch in string:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True

    return False

def init():
    lists = []
    # reading the json file
    with open('./translation2019zh/translation2019zh_train.json','r', encoding='utf-8') as dat_f:
        data = []
        for i,line in enumerate(dat_f):
            data = json.loads(line)

            if is_chinese(data['chinese']) == True:
                if len(data['chinese'])<6:
                    lists.append(data)
                if (len(lists)+1)%100 == 0:
                    break

    df = pd.DataFrame(lists)
    df.to_csv('datafile.csv', encoding='utf-8', index=False)
    

init()
df = pd.read_csv('datafile.csv')
df['chinese'] = df['chinese'].apply(lambda x: '@' + x + '。')
en_data = df.english.values.tolist()
ch_data = df.chinese.values.tolist()
en_vocab = set(''.join(en_data))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}
ch_vocab = set(''.join(ch_data))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

In [16]:
def num_data(en2id,ch2id,en_data):
    en_num_data = [[en2id[en] for en in line ] for line in en_data]
    ch_num_data = [[ch2id[ch] for ch in line] for line in ch_data]
    de_num_data = [[ch2id[ch] for ch in line][1:] for line in ch_data]
 
    max_encoder_seq_length = max([len(txt) for txt in en_num_data])
    max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
    print('max encoder length:', max_encoder_seq_length)
    print('max decoder length:', max_decoder_seq_length)

    encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
    decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
    decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

    for i in range(len(ch_num_data)):
        for t, j in enumerate(en_num_data[i]):
            encoder_input_data[i, t, j] = 1.
        for t, j in enumerate(ch_num_data[i]):
            decoder_input_data[i, t, j] = 1.
        for t, j in enumerate(de_num_data[i]):
            decoder_target_data[i, t, j] = 1.

    print('index data:\n', en_num_data[1])
    print('one hot data:\n', encoder_input_data[1])
    return encoder_input_data,decoder_input_data,decoder_target_data

nd = num_data(en2id,ch2id,en_data)

max encoder length: 75
max decoder length: 7
index data:
 [5, 41, 57, 49, 50, 28, 33, 49, 3, 24, 33, 33, 57, 33, 33, 57, 0, 49, 25, 62, 49, 28, 49, 0, 57, 51, 14, 45, 53]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [17]:
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256
LEARNING_RATE = 0.01
BATCH_SIZE = 20
EPOCHS = 250

def train():
    encoder_inputs = Input(shape=(None, EN_VOCAB_SIZE))
    encoder_h1, encoder_state_h1, encoder_state_c1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)(encoder_inputs)
    encoder_h2, encoder_state_h2, encoder_state_c2 = LSTM(HIDDEN_SIZE, return_state=True)(encoder_h1)
    decoder_inputs = Input(shape=(None, CH_VOCAB_SIZE))
    lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
    lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
    decoder_dense = Dense(CH_VOCAB_SIZE, activation='softmax')

    decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
    decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
    decoder_outputs = decoder_dense(decoder_h2)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    model.fit([nd[0], nd[1]], nd[2],batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.)
    model.save('s2s.h5')# Save model
    encoder_model = Model(encoder_inputs, [encoder_state_h1, encoder_state_c1, encoder_state_h2, encoder_state_c2])# encoder模型和訓練相同
    decoder_state_input_h1 = Input(shape=(HIDDEN_SIZE,))# 預測模型中的decoder的初始化狀態需要傳入新的狀態
    decoder_state_input_c1 = Input(shape=(HIDDEN_SIZE,))
    decoder_state_input_h2 = Input(shape=(HIDDEN_SIZE,))
    decoder_state_input_c2 = Input(shape=(HIDDEN_SIZE,))
    decoder_h1, state_h1, state_c1 = lstm1(decoder_inputs, initial_state=[decoder_state_input_h1, decoder_state_input_c1])# 使用傳入的值來初始化當前模型的輸入狀態
    decoder_h2, state_h2, state_c2 = lstm2(decoder_h1, initial_state=[decoder_state_input_h2, decoder_state_input_c2])
    decoder_outputs = decoder_dense(decoder_h2)
    decoder_model = Model([decoder_inputs, decoder_state_input_h1, decoder_state_input_c1, decoder_state_input_h2, decoder_state_input_c2], 
                        [decoder_outputs, state_h1, state_c1, state_h2, state_c2])
    return encoder_model,decoder_model

model = train()

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           [(None, None, 67)]   0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, None, 281)]  0                                            
__________________________________________________________________________________________________
lstm_14 (LSTM)                  [(None, None, 256),  331776      input_17[0][0]                   
__________________________________________________________________________________________________
lstm_16 (LSTM)                  [(None, None, 256),  550912      input_18[0][0]                   
                                                                 lstm_14[0][1]              

5/5 [==============================] - 1s 126ms/step - loss: 0.6823 - accuracy: 0.6522
Epoch 61/250
5/5 [==============================] - 1s 131ms/step - loss: 0.6840 - accuracy: 0.6508
Epoch 62/250
5/5 [==============================] - 1s 123ms/step - loss: 0.6785 - accuracy: 0.6508
Epoch 63/250
5/5 [==============================] - 1s 132ms/step - loss: 0.6785 - accuracy: 0.6522
Epoch 64/250
5/5 [==============================] - 1s 132ms/step - loss: 0.6774 - accuracy: 0.6508
Epoch 65/250
5/5 [==============================] - 1s 130ms/step - loss: 0.6755 - accuracy: 0.6508
Epoch 66/250
5/5 [==============================] - 1s 137ms/step - loss: 0.6780 - accuracy: 0.6479
Epoch 67/250
5/5 [==============================] - 1s 138ms/step - loss: 0.6785 - accuracy: 0.6522
Epoch 68/250
5/5 [==============================] - 1s 137ms/step - loss: 0.6756 - accuracy: 0.6522
Epoch 69/250
5/5 [==============================] - 1s 136ms/step - loss: 0.6741 - accuracy: 0.6522
Epoch 70/250


5/5 [==============================] - 1s 118ms/step - loss: 0.6695 - accuracy: 0.6508
Epoch 142/250
5/5 [==============================] - 1s 117ms/step - loss: 0.6626 - accuracy: 0.6508
Epoch 143/250
5/5 [==============================] - 1s 116ms/step - loss: 0.6637 - accuracy: 0.6537
Epoch 144/250
5/5 [==============================] - 1s 115ms/step - loss: 0.6619 - accuracy: 0.6465
Epoch 145/250
5/5 [==============================] - 1s 117ms/step - loss: 0.6630 - accuracy: 0.6537
Epoch 146/250
5/5 [==============================] - 1s 115ms/step - loss: 0.6615 - accuracy: 0.6508
Epoch 147/250
5/5 [==============================] - 1s 115ms/step - loss: 0.6604 - accuracy: 0.6522
Epoch 148/250
5/5 [==============================] - 1s 114ms/step - loss: 0.6615 - accuracy: 0.6522
Epoch 149/250
5/5 [==============================] - 1s 116ms/step - loss: 0.6624 - accuracy: 0.6522
Epoch 150/250
5/5 [==============================] - 1s 115ms/step - loss: 0.6635 - accuracy: 0.6522
Epoc

5/5 [==============================] - 1s 133ms/step - loss: 0.9562 - accuracy: 0.6003
Epoch 223/250
5/5 [==============================] - 1s 133ms/step - loss: 0.8971 - accuracy: 0.6032
Epoch 224/250
5/5 [==============================] - 1s 138ms/step - loss: 1.1606 - accuracy: 0.5743
Epoch 225/250
5/5 [==============================] - 1s 144ms/step - loss: 6.6292 - accuracy: 0.4776
Epoch 226/250
5/5 [==============================] - 1s 116ms/step - loss: 18.0255 - accuracy: 0.0707
Epoch 227/250
5/5 [==============================] - 1s 139ms/step - loss: 8.0263 - accuracy: 0.2121
Epoch 228/250
5/5 [==============================] - 1s 131ms/step - loss: 4.9970 - accuracy: 0.2035
Epoch 229/250
5/5 [==============================] - 1s 141ms/step - loss: 3.6591 - accuracy: 0.2641
Epoch 230/250
5/5 [==============================] - 1s 128ms/step - loss: 3.0610 - accuracy: 0.2900
Epoch 231/250
5/5 [==============================] - 1s 135ms/step - loss: 2.5807 - accuracy: 0.3232
Epo

In [18]:
def result():    
    for k in range(0,99):
        test_data = nd[0][k:k+1]
        h1, c1, h2, c2 = model[0].predict(test_data)
        target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
        target_seq[0, 0, ch2id['@']] = 1
        outputs = []
        while True:
            output_tokens, h1, c1, h2, c2 = model[1].predict([target_seq, h1, c1, h2, c2])
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            outputs.append(sampled_token_index)
            target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
            target_seq[0, 0, sampled_token_index] = 1
            if sampled_token_index == ch2id['。'] or len(outputs) > 10:
                break
        
        print(en_data[k])
        print(''.join([id2ch[i] for i in outputs]))

result()

Side-to-Side Movements.
我保证。
She was possessed by a devil.
我保证。
The majority was wrong last time.
我保证。
Great talents flower late.
我保证。
Erhai Lake （in Yunnan Province）
他不专上。
It strikes one as very strange.
我保证。
Did I divide right?
他乱摇铃。
I promise".
我保证。
All was quiet and not a soul was to be seen.
我保证。
Folder wire glass;
我保证。
Steam bath;
我保证。
Package sealing.
我保证。
Mud on salt.
他乱摇铃。
What street?
我保证。
You must be very quiet. Hold tight to me.
他乱摇铃。
Frozen ocean shrimp;
我保证。
It was he.
我保证。
Target Line;
我保证。
Chemical potential.
他乱摇铃。
He ate no soup.
他乱摇铃。
What is to be done?
我保证。
Front wheel drive.
我保证。
All streams flow into the Huanghe River.
我保证。
A:It's on the seccond floor.
我保证。
All the boats and carts started off at the same time.
我保证。
The Univ.
我保证。
The printing plant.
我保证。
It looks like rain.
我保证。
Straight tool holder .
我保证。
I'm afraid of dying a violent death.
我怕暴死。
How do they channel it?
他乱摇铃。
You love me, then?
他乱摇铃。
The fragrance of flowers assails one's nose.
我保证。
Octal Digit?